# <a id='toc1_'></a>[PCE:Couplage de Rdkit Crest-xTB avec AlGORITHME DE SCHARBER ](#toc0_)

1. **MVOTO KONGO Patrick Sorrel**, sorrel.mvoto@facsciences-uy1.cm
    * Department of Physics, Faculty of Science, University of Yaounde I 
Etudiant de Master au Laboratoire de 
    * Physique Atomique Moleculaire et Biophysique

JUIN 2024

* Nos travaux de mémoire sont basés sur le flux de travail de Tartarus. L'algorithme est le suivant :
<img src="./Tartarsus.png" width="1500"></center>

 Une illustration du cadre Tartarus, mettant en évidence les tâches de conception réelles qui sont définies et associées à des flux de travail et des ensembles de données de simulation dans Tartarus




*  Notre objectif de recherche consiste à concevoir des petites molécules dotées de propriétés électroniques spécifiques, notamment des molécules capables d'effectuer la séparation des charges, inspirées de la conception photovoltaïque organique (OPV). Nous avons deux tâches individuelles :

- Recherche d'une molécule donneuse organique à utiliser avec l'ester méthylique de l'acide l'ester méthylique de
    l'acide ${[6,6]­phényl­C61­butyrique(PCBM)}$.
- Recherche d'une molécule accepteuse à utiliser dans des dispositifs basés sur le poly[N­90­heptadécanyl­2,7­carbazole­alt­5,5­(40,70­di­2­thiényl­20,10,30­benzothiaMachine  (PCDTBT)).

*  Notre travail actuel porte sur le modèle ci-dessous :
<img src="./opv.png" width="2000"></center>

 schématique du flux de travail de simulation de propriétés pour la conception de références photovoltaïques organiques

In [1]:
import pygments
import pandas as pd

In [2]:
import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdDetermineBonds
from pyscf.data import nist

au2ev = nist.HARTREE2EV

import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdDetermineBonds
from pyscf.data import nist

au2ev = nist.HARTREE2EV

# Paramètres
input_dir = 'GDB-9'  # Dossier contenant les fichiers .xyz d'origine
output_dir = 'GDB-V2024'  # Dossier où seront créés les nouveaux fichiers .xyz
output_file = 'Patrick_GDB-9_dataframe.csv'  # Nom du fichier CSV pour le dataframe
max_files = 20885  # Nombre maximum de fichiers à traiter
df_csv = pd.read_csv('XTB_CREST_GDB9.csv')
# Créer le dossier de sortie s'il n'existe pas
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Fonction pour nettoyer les fichiers XYZ
def cleanup_GDB9_xyz(fname):
    ind = open(fname).readlines()
    nAts = int(ind[0])
    smi = ind[-2].split()[-1]
    ind[1] = '\n'
    ind = ind[:nAts+2]
    for i in range(2, nAts+2):
        l = ind[i]
        l = l.split()
        l.pop(-1)
        ind[i] = '\t'.join(l) + '\n'
    ind = ''.join(ind)
    return ind, smi 

# Initialiser la liste pour le dataframe
data = []

# Parcourir les 1000 premiers fichiers .xyz d'origine
for i in range(1, max_files + 1):
    filename = f'dsgdb9nsd_{i:06d}.xyz'
    filepath = os.path.join(input_dir, filename)

    if os.path.exists(filepath):
        # Construire le nouveau nom de fichier
        new_filename = f'GDB{i}.xyz'
        new_filepath = os.path.join(output_dir, new_filename)

        # Extraire les coordonnées XYZ, le nom du monde GDB, et les propriétés
        with open(filepath, 'r') as f:
            lines = f.readlines()
            num_atoms = int(lines[0].strip())
            world_name = lines[1].strip().split('\t')[0]
            properties = [float(x) for x in lines[1].strip().split('\t')[1:]]
            xyz_coords = [line.strip().split() for line in lines[2:2+num_atoms]]
            smiles = lines[-2].strip().split()[-2]
            smiles_12 = lines[-2].strip().split()[-1]

        # Utiliser la fonction pour nettoyer et obtenir les coordonnées XYZ
        ind, smi = cleanup_GDB9_xyz(filepath)

        # Écrire les nouvelles coordonnées XYZ dans le nouveau fichier
        with open(new_filepath, 'w') as f:
            f.write(f'{num_atoms}\n')
            f.write('\n')
            for row in xyz_coords:
                formatted_row = '{:<2} {:>15} {:>15} {:>15}\n'.format(row[0], row[1], row[2], row[3])
                f.write(formatted_row)
            mol = Chem.MolFromSmiles(smiles)
            homo = properties[5]
            lumo = properties[6]
            gap = properties[7]
            # Récupérer les informations correspondantes dans le dataframe
            if world_name in df_csv['smiles_key'].values:
                # Récupérer les informations correspondantes dans le dataframe
                
                df_csv.at[i,'smiles'] = smiles
                df_csv.at[i,'HOMO(eV)']  = homo * au2ev,
                df_csv.at[i,'LUMO(eV)'] = lumo * au2ev,
                df_csv.at[i,'GAP(eV)'] =gap * au2ev

            # Ajouter les informations au dataframe
            data.append({
                'smiles_key': world_name,
                'smiles': smiles,
                'smiles_c': smiles_12,
                'mol': mol,
                'HOMO(HARTREE)': homo,
                'LUMO(HARTREE)': lumo,
                'gap(HARTREE)': gap,
                'HOMO(eV)': homo * au2ev,
                'LUMO(eV)': lumo * au2ev,
                'gap(eV)': gap * au2ev,
            })

# Créer le dataframe et l'enregistrer dans un fichier CSV
properties_df = pd.DataFrame(data)
# Remarque : RDKit Mol objects ne peuvent pas être directement enregistrés dans un fichier CSV. 
# Vous pouvez soit les convertir en un format sérialisé, soit les omettr
properties_df.drop(columns=['mol']).to_csv(output_file, index=False)

df1=df_csv.iloc[0:19755,:]
df1.to_csv('my_datasets.csv', index=False)

df1=pd.read_csv('Patrick_GDB-9_dataframe.csv')
#counter = 0
df1=df1[(df1['HOMO(eV)']< 0) &(df1['LUMO(eV)']<0)]
# # Utiliser une boucle for pour filtrer les données et les ajouter à la nouvelle DataFrame
# for i in range(len(df1)):
#     df=df1.loc[i, 'HOMO(eV)'] < 0 and df1.loc[i, 'LUMO(eV)'] < 0
#     df.loc[counter] = df1.loc[i]
#     counter += 1

# # Couper la DataFrame à la taille réelle des données filtrées
# df = df[:counter]

# # Afficher la nouvelle DataFrame filtrée
# print(df)
df1.to_csv('Patrick_GDB-9_datasets.csv', index=False)

In [3]:
df=pd.read_csv('XTB_CREST_GDB9.csv')
df

,Unnamed: 0,smiles,mol,HOMO(eV),LUMO(eV),GAP(eV),HOMO_xtb(eV),LUMO_xtb(eV),GAP_xtb(eV)
0,gdb 1,C,<rdkit.Chem.rdchem.Mol object at 0x788d81a32e40>,-10.549854,3.186453,13.736308,-12.7202,4.6329,17.353095
1,gdb 2,N,<rdkit.Chem.rdchem.Mol object at 0x788d81a32dd0>,-6.993326,2.255824,9.249150,-10.5031,1.7297,12.232824
2,gdb 3,O,<rdkit.Chem.rdchem.Mol object at 0x788d81a3bc10>,-7.967494,1.869422,9.836916,-12.1467,2.2396,14.386316
3,gdb 4,C#C,<rdkit.Chem.rdchem.Mol object at 0x788d81a3bba0>,-7.741639,1.376896,9.118535,-11.5117,-4.2239,7.287776
4,gdb 5,C#N,<rdkit.Chem.rdchem.Mol object at 0x788d81a32c80>,-9.806984,0.519737,10.329442,-12.2419,-6.2921,5.949769
...,...,...,...,...,...,...,...,...,...
19420,gdb 19752,CC1C2C3CC1CC23,<rdkit.Chem.rdchem.Mol object at 0x788d8156ff90>,-7.028701,2.517053,9.545754,-10.7227,-1.3496,9.373166
19421,gdb 19753,CC1C2C3CC1CN23,<rdkit.Chem.rdchem.Mol object at 0x788d8156fa50>,-6.334811,2.402765,8.737576,-10.0806,-1.8878,8.192813
19422,gdb 19754,CC1C2C3CC1OC23,<rdkit.Chem.rdchem.Mol object at 0x788d8156f270>,-6.190590,2.530659,8.721249,-10.3509,-1.7017,8.649156
19423,gdb 19755,CC1C2C3OC1OC23,<rdkit.Chem.rdchem.Mol object at 0x788d8156f040>,-6.498079,2.473515,8.971594,-10.6935,-2.0800,8.613450


In [4]:
df=df.iloc[0:1000,:]
df

,Unnamed: 0,smiles,mol,HOMO(eV),LUMO(eV),GAP(eV),HOMO_xtb(eV),LUMO_xtb(eV),GAP_xtb(eV)
0,gdb 1,C,<rdkit.Chem.rdchem.Mol object at 0x788d81a32e40>,-10.549854,3.186453,13.736308,-12.7202,4.6329,17.353095
1,gdb 2,N,<rdkit.Chem.rdchem.Mol object at 0x788d81a32dd0>,-6.993326,2.255824,9.249150,-10.5031,1.7297,12.232824
2,gdb 3,O,<rdkit.Chem.rdchem.Mol object at 0x788d81a3bc10>,-7.967494,1.869422,9.836916,-12.1467,2.2396,14.386316
3,gdb 4,C#C,<rdkit.Chem.rdchem.Mol object at 0x788d81a3bba0>,-7.741639,1.376896,9.118535,-11.5117,-4.2239,7.287776
4,gdb 5,C#N,<rdkit.Chem.rdchem.Mol object at 0x788d81a32c80>,-9.806984,0.519737,10.329442,-12.2419,-6.2921,5.949769
...,...,...,...,...,...,...,...,...,...
995,gdb 1022,N#CC1=CNC=N1,<rdkit.Chem.rdchem.Mol object at 0x788db800e4a0>,-6.906250,-0.541507,6.364743,-10.9542,-6.9272,4.027010
996,gdb 1023,N#CC1=COC=C1,<rdkit.Chem.rdchem.Mol object at 0x788db800e2e0>,-6.987884,-0.892533,6.095350,-11.4673,-7.1812,4.286087
997,gdb 1024,N#CC1=COC=N1,<rdkit.Chem.rdchem.Mol object at 0x788db800e510>,-7.548438,-1.287099,6.264061,-11.7120,-7.6724,4.039603
998,gdb 1025,O=CC1=CC=CN1,<rdkit.Chem.rdchem.Mol object at 0x788db800e970>,-6.242292,-1.172811,5.069481,-10.8325,-7.5870,3.245417


In [5]:
#import xtb

In [6]:
!xtb --version

      -----------------------------------------------------------      
     |                   =====================                   |     
     |                           x T B                           |     
     |                   =====================                   |     
     |                         S. Grimme                         |     
     |          Mulliken Center for Theoretical Chemistry        |     
     |                    University of Bonn                     |     
      -----------------------------------------------------------      

   * xtb version 6.6.1 (8d0f1dd) compiled by 'conda@1efc2f54142f' on 2023-08-01

normal termination of xtb


In [10]:
import os, sys
import inspect
from pathlib import Path
import tempfile
from utils import run_command

import rdkit
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import RDConfig
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer

import numpy as np
import torch
import torch.nn as nn

In [11]:
MY_NEW_GDB9_DATA = os.path.join(os.getcwd(), 'MY_NEW_GDB9_DATA')
os.makedirs(MY_NEW_GDB9_DATA, exist_ok=True)


#      Modele de sharber
####  <a id='toc1_3_'></a>[Utilisation de crest et xTB pour la recherches des conformers](#toc0_)
 <center> <img src = "./crest.jpeg" width = "600">
 <img src = "./xtb.jpeg" width = "600"> </center> 

df.to_csv('test.csv', index=False) 

df=pd.read_csv('test.csv')
df

import os
import subprocess
from rdkit import Chem
import pandas as pd

def run_command(command, verbose=0):
    result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if verbose > 0:
        print(result.stdout.decode())
        if verbose > 1:
            print(result.stderr.decode())
    return result.returncode, result.stdout.decode(), result.stderr.decode()

def calculate_properties(df, working_dir):
    """
    Calculates molecular properties for SMILES strings in a DataFrame,
    handling xtb convergence errors and filtering unsuccessful molecules.

    Args:
        df (pandas.DataFrame): A DataFrame containing a "smiles" column.
        working_dir (str): The directory to use for calculations and temporary files.

    Returns:
        pandas.DataFrame: A DataFrame containing calculated properties for successful molecules.
    """
    dtb=[]
    filtered_df = df.copy()  # Create a copy to avoid modifying the original DataFrame
    successful_results = []  # List to store results for successful molecules

    for i in range(len(filtered_df)):
        smiles = filtered_df.loc[i, "smiles"]
        smiles_key = str(filtered_df.loc[i, "Unnamed: 0"])
         
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)
        charge = Chem.rdmolops.GetFormalCharge(mol)
        atom_number = mol.GetNumAtoms()

        # Create a single directory for all calculations
        directory = os.path.join(working_dir, smiles_key)
        os.makedirs(directory, exist_ok=True)

        # Change directory for calculations
        os.chdir(directory)

        # Write the smile to a file
        with open('test.smi', 'w') as f:
            f.write(smiles)

        # Prepare the input file
        os.system('obabel test.smi --gen3D -O test.xyz')

        # Run the preliminary xtb
        command_pre = f'CHARGE={charge}; xtb test.xyz --gfn 2 --opt vtight -c $CHARGE --iterations 4000'
       
        try:
            # Run xtb with error handling
            result, stdout, stderr = run_command(command_pre, verbose=1)
            if result != 0 or "Initial geometry optimization failed!" in stderr:
                print(f"xtb preliminary optimization failed for '{smiles}'. Skipping.")
                filtered_df.drop(i, inplace=True)  # Remove molecule from DataFrame
                continue  # Skip to the next molecule
        except Exception as e:
            print(f"An error occurred while running xtb for '{smiles}': {e}")
            filtered_df.drop(i, inplace=True)  # Remove molecule from DataFrame
            continue  # Skip to the next molecule

        os.system("rm ./gfnff_charges ./gfnff_topo")  # Optional removal
         
        # Run crest conformer ensemble
        command_crest = f'CHARGE={charge}; crest xtbopt.xyz -gff -mquick -chrg $CHARGE --noreftopo'
        os.system(command_crest)
       
        os.system('rm ./gfnff_charges ./gfnff_topo')  # Optional removal
        os.system(f'head -n {atom_number+2} crest_conformers.xyz > crest_best.xyz')

        # Run the calculation
        command = f'CHARGE={charge}; xtb crest_best.xyz --opt normal -c $CHARGE --iterations 4000 > out_dump'

        try:
            # Run xtb with error handling
            result, stdout, stderr = run_command(command, verbose=5)
            if result != 0 or "Initial geometry optimization failed!" in stderr:
                print(f"xtb final optimization failed for '{smiles}'. Skipping.")
                filtered_df.drop(i, inplace=True)  # Remove molecule from DataFrame
                continue  # Skip to the next molecule
        except Exception as e:
            print(f"An error occurred while running xtb for '{smiles}': {e}")
            filtered_df.drop(i, inplace=True)  # Remove molecule from DataFrame
            continue  # Skip to the next molecule

        # Process and store the result if successful (implementation details omitted)
        # For example, parse output and add to results

        with open('./out_dump', 'r') as f:
            text_content = f.readlines()
        output_index = [i for i in range(len(text_content)) if 'Property Printout' in text_content[i]]
        text_content = text_content[output_index[0]:]
        homo_data = [x for x in text_content if '(HOMO)' in x]
        lumo_data = [x for x in text_content if '(LUMO)' in x]
        homo_lumo_gap = [x for x in text_content if 'HOMO-LUMO GAP' in x]
        mol_dipole = [text_content[i:i+4] for i, x in enumerate(text_content) if 'molecular dipole:' in x]
        lumo_val = float(lumo_data[0].split(' ')[-2])
        homo_val = float(homo_data[0].split(' ')[-2])
        homo_lumo_val = float(homo_lumo_gap[0].split(' ')[-5])
        mol_dipole_val = float(mol_dipole[0][-1].split(' ')[-1])

        # Write the properties to a single file (modify as needed)
        with open(os.path.join(directory, 'properties.txt'), 'a') as f:
            f.write(f'SMILES: {smiles}\n')
            f.write(f'LUMO: {lumo_val}\n')
            f.write(f'HOMO: {homo_val}\n')
            f.write(f'HOMO-LUMO GAP: {homo_lumo_val}\n')
        dtb.append([homo_lumo_val, mol_dipole_val, homo_val, lumo_val])

        
        os.chdir('..')

    df_xtb = pd.DataFrame(dtb, columns=yesso)
    return df_xtb

df_xtb = calculate_properties(df, MY_NEW_GDB9_DATA)

In [12]:
import os
import subprocess
from contextlib import contextmanager, redirect_stderr, redirect_stdout
from os import devnull
from rdkit import Chem
import pandas as pd

@contextmanager
def suppress_output(verbose):
    """Suppress output when verbose is False"""
    if verbose:
        yield
    else:
        with open(devnull, 'w') as fnull:
            with redirect_stderr(fnull), redirect_stdout(fnull):
                yield

def run_command(command, verbose):
    if verbose:
        result = subprocess.run(command, shell=True)
    else:
        result = subprocess.run(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return result.returncode

def calculate_properties(df, working_dir, verbose=0):
    """
    Calculates molecular properties for SMILES strings in a DataFrame,
    handling xtb convergence errors and filtering unsuccessful molecules.

    Args:
        df (pandas.DataFrame): A DataFrame containing a "smiles" column.
        working_dir (str): The directory to use for calculations and temporary files.
        verbose (int): Verbosity level (0: no output, 1: output)

    Returns:
        pandas.DataFrame: A DataFrame containing calculated properties for successful molecules.
    """
    
    dtb = []
    yesso = ["GAP_NEW_xtb(eV)", "MO_NEWS_DIP_xtb(eV)", "HOMO_NEW_xtb(eV)", "LUMO__NEW_xtb(eV)"]

    filtered_df = df.copy()  # Create a copy to avoid modifying the original DataFrame

    for i in range(len(filtered_df)):
        smiles = filtered_df.loc[i, "smiles"]
        smiles_key = filtered_df.loc[i, "Unnamed: 0"]
         
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)
        charge = Chem.rdmolops.GetFormalCharge(mol)
        atom_number = mol.GetNumAtoms()

        # Create a single directory for all calculations
        directory = os.path.join(working_dir, smiles_key)
        os.makedirs(directory, exist_ok=True)

        # Change directory for calculations
        os.chdir(directory)

        # Write the SMILES to a file
        with open('test.smi', 'w') as f:
            f.write(smiles)

        # Prepare the input file
        os.system('obabel test.smi --gen3D -O test.xyz')

        # Run the preliminary xtb
        command_pre = f'CHARGE={charge}; xtb test.xyz --gfn 2 --opt vtight -c $CHARGE --iterations 4000'

        with suppress_output(verbose):
            try:
                # Run xtb with error handling
                result = run_command(command_pre, verbose)
                if result != 0:
                    print(f"xtb preliminary optimization failed for '{smiles}'. Skipping.")
                    filtered_df.drop(i, inplace=True)  # Remove molecule from DataFrame
                    continue  # Skip to the next molecule
            except Exception as e:
                print(f"An error occurred while running xtb for '{smiles}': {e}")
                filtered_df.drop(i, inplace=True)  # Remove molecule from DataFrame
                continue  # Skip to the next molecule

        os.system("rm -f ./gfnff_charges ./gfnff_topo")  # Optional removal

        # Run crest conformer ensemble
        command_crest = f'CHARGE={charge}; crest xtbopt.xyz -gff -mquick -chrg $CHARGE --noreftopo'

        with suppress_output(verbose):
            try:
                result = run_command(command_crest, verbose)
                if result != 0:
                    print(f"Crest conformer search failed for '{smiles}'. Skipping.")
                    filtered_df.drop(i, inplace=True)  # Remove molecule from DataFrame
                    continue  # Skip to the next molecule
            except Exception as e:
                print(f"An error occurred while running crest for '{smiles}': {e}")
                filtered_df.drop(i, inplace=True)  # Remove molecule from DataFrame
                continue  # Skip to the next molecule

        # Check if crest_conformers.xyz exists
        if not os.path.exists('crest_conformers.xyz'):
            print(f"Conformer file not found for '{smiles}'. Skipping.")
            filtered_df.drop(i, inplace=True)  # Remove molecule from DataFrame
            continue  # Skip to the next molecule

        os.system(f'head -n {atom_number + 2} crest_conformers.xyz > crest_best.xyz')

        # Run the final
                # Run the calculation
        command = 'CHARGE={};xtb {} --opt normal -c $CHARGE --iterations 4000 > out_dump'.format(charge, 'crest_best.xyz')

        try:
            # Run xtb with error handling
            result = os.system(command)
            if result != 0:
                print(f"xtb final optimization failed for '{smiles}'. Skipping.")
                filtered_df.drop(i, inplace=True)  # Remove molecule from DataFrame
                continue  # Skip to the next molecule
        except Exception as e:
            print(f"An error occurred while running xtb for '{smiles}': {e}")
            filtered_df.drop(i, inplace=True)  # Remove molecule from DataFrame
            continue  # Skip to the next molecule

        # Read the output (implementation details omitted)
        # Read the output
        with open('./out_dump', 'r') as f:
            text_content = f.readlines()
        output_index = [i for i in range(len(text_content)) if 'Property Printout' in text_content[i]]
        text_content = text_content[output_index[0]:]
        homo_data = [x for x in text_content if '(HOMO)' in x]
        lumo_data = [x for x in text_content if '(LUMO)' in x]
        homo_lumo_gap = [x for x in text_content if 'HOMO-LUMO GAP' in x]
        mol_dipole = [text_content[i:i+4] for i, x in enumerate(text_content) if 'molecular dipole:' in x]
        lumo_val = float(lumo_data[0].split(' ')[-2])
        homo_val = float(homo_data[0].split(' ')[-2])
        homo_lumo_val = float(homo_lumo_gap[0].split(' ')[-5])
        mol_dipole_val = float(mol_dipole[0][-1].split(' ')[-1])






        # Write the properties to a single file (modify as needed)
        with open(os.path.join(directory, 'properties.txt'), 'a') as f:
            f.write(f'SMILES: {smiles}\n')
            f.write(f'LUMO: {lumo_val}\n')
            f.write(f'HOMO: {homo_val}\n')
            f.write(f'HOMO-LUMO GAP: {homo_lumo_val}\n')
        dtb.append([homo_lumo_val, mol_dipole_val, homo_val, lumo_val])

        
        os.chdir('..')

    df_xtb = pd.DataFrame(dtb, columns=yesso)
    return df_xtb

df_xtb = calculate_properties(df, MY_NEW_GDB9_DATA)

1 molecule converted
normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
1 molecule converted
normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
1 molecule converted
normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
1 molecule converted
normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
1 molecule converted
normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
1 molecule converted
normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
1 molecule converted
normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
1 molecule converted
normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
1 molecule conve

import os
import pandas as pd
from rdkit import Chem


def calculate_properties(df, working_dir):
    """
    Calculates molecular properties for SMILES strings in a DataFrame,
    handling xtb convergence errors and filtering unsuccessful molecules.

    Args:
        df (pandas.DataFrame): A DataFrame containing a "smiles" column.
        working_dir (str): The directory to use for calculations and temporary files.

    Returns:
        pandas.DataFrame: A DataFrame containing calculated properties for successful molecules.
    """

    dtb = []
    yesso = ["GAP_NEW_xtb(eV)","MO_NEWS_DIP_xtb (eV)","HOMO_NEW_xtb(eV)","LUMO__NEW_xtb(eV)"]

    filtered_df = df.copy()  # Create a copy to avoid modifying the original DataFrame

    for i in range(len(filtered_df)):
        smiles = filtered_df.loc[i, "smiles"]
        smiles_key = filtered_df.loc[i, "Unnamed: 0"]
         
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)
        charge = Chem.rdmolops.GetFormalCharge(mol)
        atom_number = mol.GetNumAtoms()

        # Create a single directory for all calculations
        directory = os.path.join(working_dir, smiles_key )
        os.makedirs(directory, exist_ok=True)

        # Change directory for calculations
        os.chdir(directory)
        system = lambda x: run_command(x, verbose=5)

        # Write the smile to a file
        with open('test.smi', 'w') as f:
            f.write(smiles)

        # Prepare the input file
        os.system('obabel test.smi --gen3D -O test.xyz')

        # Run the preliminary xtb: 
        command_pre = 'CHARGE={};xtb {} --gfn 2 --opt vtight -c $CHARGE --iterations 4000'.format(charge, 'test.xyz')
       
        try:
            # Run xtb with error handling
            result = os.system(command_pre)
            if result != 0:
                print(f"xtb preliminary optimization failed for '{smiles}'. Skipping.")
                filtered_df.drop(i, inplace=True)  # Remove molecule from DataFrame
                continue  # Skip to the next molecule
        except Exception as e:
            print(f"An error occurred while running xtb for '{smiles}': {e}")
            filtered_df.drop(i, inplace=True)  # Remove molecule from DataFrame
            continue  # Skip to the next molecule
            

        os.system("rm ./gfnff_charges ./gfnff_topo")  # Optional removal
         
        # Run crest conformer ensemble
        command_crest = 'CHARGE={};crest {} -gff -mquick -chrg $CHARGE --noreftopo'.format(charge, 'xtbopt.xyz')
        os.system(command_crest)
       
        os.system('rm ./gfnff_charges ./gfnff_topo')  # Optional removal
        os.system('head -n {} crest_conformers.xyz > crest_best.xyz'.format(atom_number+2))

        # Run the calculation
        command = 'CHARGE={};xtb {} --opt normal -c $CHARGE --iterations 4000 > out_dump'.format(charge, 'crest_best.xyz')

        try:
            # Run xtb with error handling
            result = os.system(command)
            if result != 0:
                print(f"xtb final optimization failed for '{smiles}'. Skipping.")
                filtered_df.drop(i, inplace=True)  # Remove molecule from DataFrame
                continue  # Skip to the next molecule
        except Exception as e:
            print(f"An error occurred while running xtb for '{smiles}': {e}")
            filtered_df.drop(i, inplace=True)  # Remove molecule from DataFrame
            continue  # Skip to the next molecule

        # Read the output (implementation details omitted)
        # Read the output
        with open('./out_dump', 'r') as f:
            text_content = f.readlines()
        output_index = [i for i in range(len(text_content)) if 'Property Printout' in text_content[i]]
        text_content = text_content[output_index[0]:]
        homo_data = [x for x in text_content if '(HOMO)' in x]
        lumo_data = [x for x in text_content if '(LUMO)' in x]
        homo_lumo_gap = [x for x in text_content if 'HOMO-LUMO GAP' in x]
        mol_dipole = [text_content[i:i+4] for i, x in enumerate(text_content) if 'molecular dipole:' in x]
        lumo_val = float(lumo_data[0].split(' ')[-2])
        homo_val = float(homo_data[0].split(' ')[-2])
        homo_lumo_val = float(homo_lumo_gap[0].split(' ')[-5])
        mol_dipole_val = float(mol_dipole[0][-1].split(' ')[-1])






        # Write the properties to a single file (modify as needed)
        with open(os.path.join(directory, 'properties.txt'), 'a') as f:
            f.write(f'SMILES: {smiles}\n')
            f.write(f'LUMO: {lumo_val}\n')
            f.write(f'HOMO: {homo_val}\n')
            f.write(f'HOMO-LUMO GAP: {homo_lumo_val}\n')
        dtb.append([homo_lumo_val, mol_dipole_val, homo_val, lumo_val])

        
        os.chdir('..')

    df_xtb = pd.DataFrame(dtb, columns=yesso)
    return df_xtb

df_xtb = calculate_properties(df, MY_NEW_GDB9_DATA)

In [ ]:
df2=df_xtb
df2

,GAP_xtb(eV),MO_DIP_xtb (eV),HOMO_xtb(eV),LUMO_xtb(eV)
0,3.620171,2.335,-11.5341,-7.9140
1,3.891062,2.938,-11.1332,-7.2422
2,4.108607,3.399,-10.8443,-6.7357
3,4.882738,0.000,-10.9033,-6.0205
4,4.526436,3.699,-11.9696,-7.4432
...,...,...,...,...
95,1.000973,2.045,-11.4683,-10.4673
96,5.335380,4.157,-12.0775,-6.7421
97,4.567283,2.140,-10.7174,-6.1502
98,4.293110,3.373,-11.3260,-7.0329


In [ ]:
df_dFT=pd.concat([df,df_xtb ], axis=1)
df_dFT

,smiles_key,smiles,smiles_c,HOMO(HARTREE),LUMO(HARTREE),gap(HARTREE),HOMO(eV),LUMO(eV),gap(eV),GAP_xtb(eV),MO_DIP_xtb (eV),HOMO_xtb(eV),LUMO_xtb(eV)
0,gdb 6,C=O,C=O,-0.2670,-0.0406,0.2263,-7.265440,-1.104782,6.157937,3.620171,2.335,-11.5341,-7.9140
1,gdb 11,CC=O,CC=O,-0.2540,-0.0198,0.2342,-6.911692,-0.538785,6.372907,3.891062,2.938,-11.1332,-7.2422
2,gdb 18,CC(C)=O,CC(=O)C,-0.2431,-0.0087,0.2344,-6.615088,-0.236739,6.378349,4.108607,3.399,-10.8443,-6.7357
3,gdb 23,C#CC#C,C(#C)C#C,-0.2599,-0.0214,0.2386,-7.072239,-0.582324,6.492637,4.882738,0.000,-10.9033,-6.0205
4,gdb 24,C#CC#N,C(#C)C#N,-0.3102,-0.0543,0.2559,-8.440972,-1.477578,6.963394,4.526436,3.699,-11.9696,-7.4432
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,gdb 326,O=CC(=O)C=O,O=CC(=O)C=O,-0.2606,-0.1455,0.1151,-7.091287,-3.959257,3.132031,1.000973,2.045,-11.4683,-10.4673
96,gdb 329,CC(C#N)C#N,CC(C#N)C#N,-0.3433,-0.0129,0.3304,-9.341669,-0.351027,8.990642,5.335380,4.157,-12.0775,-6.7421
97,gdb 330,NC(C#C)C#N,N[C@@H](C#C)C#N,-0.2798,-0.0030,0.2768,-7.613746,-0.081634,7.532112,4.567283,2.140,-10.7174,-6.1502
98,gdb 331,NC(C#N)C#N,NC(C#N)C#N,-0.3037,-0.0279,0.2758,-8.264098,-0.759198,7.504900,4.293110,3.373,-11.3260,-7.0329


In [ ]:
df4 = pd.DataFrame()
for i in range(len(df)):
    dif_gap = abs(df2.at[i,"GAP_xtb(eV)"] - df.at[i, 'gap(eV)'])
    df4.at[i,'dif_gap(kcal/mol)'] = dif_gap  * 627.5

In [ ]:
DF_dFT=pd.concat([df_dFT, df4], axis=1)

In [ ]:
DF_dFT

,smiles_key,smiles,smiles_c,HOMO(HARTREE),LUMO(HARTREE),gap(HARTREE),HOMO(eV),LUMO(eV),gap(eV),GAP_xtb(eV),MO_DIP_xtb (eV),HOMO_xtb(eV),LUMO_xtb(eV),dif_gap(kcal/mol)
0,gdb 6,C=O,C=O,-0.2670,-0.0406,0.2263,-7.265440,-1.104782,6.157937,3.620171,2.335,-11.5341,-7.9140,1592.448032
1,gdb 11,CC=O,CC=O,-0.2540,-0.0198,0.2342,-6.911692,-0.538785,6.372907,3.891062,2.938,-11.1332,-7.2422,1557.357202
2,gdb 18,CC(C)=O,CC(=O)C,-0.2431,-0.0087,0.2344,-6.615088,-0.236739,6.378349,4.108607,3.399,-10.8443,-6.7357,1424.263086
3,gdb 23,C#CC#C,C(#C)C#C,-0.2599,-0.0214,0.2386,-7.072239,-0.582324,6.492637,4.882738,0.000,-10.9033,-6.0205,1010.211338
4,gdb 24,C#CC#N,C(#C)C#N,-0.3102,-0.0543,0.2559,-8.440972,-1.477578,6.963394,4.526436,3.699,-11.9696,-7.4432,1529.190679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,gdb 326,O=CC(=O)C=O,O=CC(=O)C=O,-0.2606,-0.1455,0.1151,-7.091287,-3.959257,3.132031,1.000973,2.045,-11.4683,-10.4673,1337.238353
96,gdb 329,CC(C#N)C#N,CC(C#N)C#N,-0.3433,-0.0129,0.3304,-9.341669,-0.351027,8.990642,5.335380,4.157,-12.0775,-6.7421,2293.676682
97,gdb 330,NC(C#C)C#N,N[C@@H](C#C)C#N,-0.2798,-0.0030,0.2768,-7.613746,-0.081634,7.532112,4.567283,2.140,-10.7174,-6.1502,1860.429740
98,gdb 331,NC(C#N)C#N,NC(C#N)C#N,-0.3037,-0.0279,0.2758,-8.264098,-0.759198,7.504900,4.293110,3.373,-11.3260,-7.0329,2015.398360


In [ ]:
DF_dFT.to_csv('my_XTB_CREST_GDB9.csv', index=False)   # Save with index